In [4]:
import requests
import json
import numpy as np
import pandas as pd

def yelp_api_request(term, location, query_number):
    """term = 'nightlife' (string)
        location = 'zipcode', 'neighborhood' (string)
        query_number = round of query (1,2,3...) -> determines offset (integer)
    """
    offset = (query_number - 1)*50 + 1 
    api_key='MYTOKEN'

    url='https://api.yelp.com/v3/businesses/search'
    params={'term': term, 'location': location, 'limit': 50, 'offset': offset}
    headers = {'Authorization': 'Bearer %s' % api_key} 
    req = requests.get(url, params=params, headers=headers)
    json_data = req.json()
    # with open('/Users/cmvandev/Desktop/debug/data.json', 'w', encoding='utf-8') as f:
    #    json.dump(json_data, f, ensure_ascii=False, indent=4) 
    # data = json.load(open('/Users/cmvandev/Desktop/debug/data.json'))
    df = pd.DataFrame(json_data["businesses"])
    business_id = list(df['id'])
    reviews_dataframe = pd.DataFrame()
    list_of_reviews = []
    new = pd.DataFrame(index=business_id, columns = ['rev1', 'rev2', 'rev3'])
    for id_ in business_id:
        list_of_reviews_for_each_business =[]
        url="https://api.yelp.com/v3/businesses/" + id_ + "/reviews"
        req_reviews = requests.get(url, headers=headers)
        #json_data_reviews = req_reviews.json()
        try:
            if req_reviews.json()['reviews'][0]['text']!= None and req_reviews.json()['reviews'][0]['text'] != '' :
                list_of_reviews_for_each_business.append(req_reviews.json()['reviews'][0]['text'])
        except:
            list_of_reviews_for_each_business.append('No Review')

        try:
            if req_reviews.json()['reviews'][1]['text']!= None and req_reviews.json()['reviews'][1]['text'] != '' :
                list_of_reviews_for_each_business.append(req_reviews.json()['reviews'][1]['text'])
        except:
            list_of_reviews_for_each_business.append('No Review')

        try:
            if req_reviews.json()['reviews'][2]['text']!= None and req_reviews.json()['reviews'][2]['text'] != '' :
                list_of_reviews_for_each_business.append(req_reviews.json()['reviews'][2]['text'])
        except:
            list_of_reviews_for_each_business.append('No Review')

        list_of_reviews.append(list_of_reviews_for_each_business)
    #     list_of_reviews_for_each_business.clear()

        new.loc[id_]['rev1'] = list_of_reviews_for_each_business[0]
        new.loc[id_]['rev2'] = list_of_reviews_for_each_business[1]
        new.loc[id_]['rev3'] = list_of_reviews_for_each_business[2]

    # merge business info and reviews dataframe on business ID
    df_merged = df.merge(new, left_on = 'id', right_index = True)

    file_name = '{}_{}_{}_yelp.csv'.format(term, location, query_number)

    #df_merged.to_csv('/Users/cmvandev/Desktop/debug/data_merged.csv')
    df_merged.to_csv('/Users/andrewnorris/capstone_legaloccupancy/datasets/api/yelp/'+file_name)

In [27]:
yelp_api_request('nightlife','10002', 16)

KeyboardInterrupt: 

NameError: name 'req' is not defined